In [4]:
#Hay que sacar datos de un pdf usando py 
import os 
import PyPDF2
import pandas as pd
from docx import Document
import pdfplumber

## **EXTRAER DATOS DE UN PDF**

In [ ]:
# Función para hacer columnas únicas
def hacer_columnas_unicas(columnas):
    nuevas = []
    contador = {}
    for col in columnas:
        if col not in contador:
            contador[col] = 0
            nuevas.append(col)
        else:
            contador[col] += 1
            nuevas.append(f"{col}_{contador[col]}")
    return nuevas

carpeta_pdfs = 'PDFs'
lista_dfs = []

for archivo in os.listdir(carpeta_pdfs):
    if archivo.endswith(".pdf"):
        ruta_pdf = os.path.join(carpeta_pdfs, archivo)
        with pdfplumber.open(ruta_pdf) as pdf:
            pagina = pdf.pages[0]
            tablas = pagina.extract_tables()
            if tablas:
                tabla = tablas[0]

                # Verificamos que hay al menos: fila de título, encabezados, y una fila de datos
                if len(tabla) > 2:
                    # Encabezados están en fila 1 (después de "Key Statistics")
                    periodos = tabla[1][1:]  # Saltamos la primera celda (vacía o título de métrica)
                    periodos = [p.replace('\n', ' ').strip() for p in periodos]

                    # Extraemos las filas de datos
                    datos = tabla[2:]

                    # Preparamos un DataFrame temporal con todas las métricas como filas
                    df_temp = pd.DataFrame(datos)

                    # Primera columna es el nombre de la métrica
                    metricas = df_temp.iloc[:, 0].tolist()

                    # Las demás columnas son los valores por periodo
                    valores = df_temp.iloc[:, 1:].T  # Transponemos: filas -> periodos
                    valores.columns = metricas
                    valores.insert(0, 'Periodo', periodos)
                    valores['origen_pdf'] = archivo

                    lista_dfs.append(valores)
                else:
                    print(f"Tabla muy pequeña en {archivo}")
            else:
                print(f"No se encontró tabla en {archivo}")

# Unimos todos los resultados
if lista_dfs:
    df_final = pd.concat(lista_dfs, ignore_index=True)
    display(df_final)
else:
    print("No se extrajeron tablas de ningún PDF.")

df_final.to_excel('resultado.xlsx')

,Periodo,Total Revenue,Growth Over Prior Year,Gross Profit Margin %,EBITDA Margin %,EBIT Margin %,Net Income Margin %,Diluted EPS Excl. Extra Items,origen_pdf,Diluted EPS Excl. Extra Items...
0,12 Months Dec-31-2023A,350.30,(64.7%),(14.1%),(18.3%),(41.3%),(32.6%),(0.44),AustralGroupSAABVLAUSTRAC1_PublicCompany (1).pdf,NaN
1,12 Months Dec-31-2024A,753.66,115.1%,38.8%,34.7%,24.1%,15.0%,0.44,AustralGroupSAABVLAUSTRAC1_PublicCompany (1).pdf,NaN
2,12 Months Mar-31-2025A,940.68,153.3%,36.7%,31.7%,23.3%,14.7%,0.53,AustralGroupSAABVLAUSTRAC1_PublicCompany (1).pdf,NaN
3,12 Months Dec-31-2023A,"336,665.00",(16.0%),32.5%,19.7%,14.6%,10.7%,8.89,ExxonMobilCorporationNYSEXOM_PublicCompany.pdf,(33.0%)
4,12 Months Dec-31-2024A,"340,568.00",1.2%,30.7%,18.9%,12.2%,9.9%,7.84,ExxonMobilCorporationNYSEXOM_PublicCompany.pdf,(11.8%)
5,12 Months Mar-31-2025A,"341,088.00",2.2%,30.8%,19.0%,12.1%,9.7%,7.55,ExxonMobilCorporationNYSEXOM_PublicCompany.pdf,(7.3%)
6,12 Months Dec-31-2025E,"328,956.22",(5.9%),52.5%,20.4%,11.9%,8.4%,6.54,ExxonMobilCorporationNYSEXOM_PublicCompany.pdf,(16.1%)
7,12 Months Dec-31-2026E,"331,927.32",0.9%,55.9%,21.9%,12.3%,9.3%,7.74,ExxonMobilCorporationNYSEXOM_PublicCompany.pdf,18.3%
8,12 Months Dec-31-2023A,"28,985.00",14.8%,23.2%,8.3%,7.9%,2.3%,0.22,MapfreSABMEMAP_PublicCompany.pdf,19.8%
9,12 Months Dec-31-2024A,"29,243.70",0.9%,24.7%,9.7%,9.3%,3.3%,0.32,MapfreSABMEMAP_PublicCompany.pdf,42.7%


In [3]:
# Guardar en Excel
#df.to_excel("tabla_extraida.xlsx", index=False)

## **EXTRAER DATOS DE UNA PAGINA WEB**

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [ ]:
def scrape_treasury_yield_curve(year):
    url = f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value={year}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')

    if table is None:
        print(f"No data found for {year}")
        return pd.DataFrame()

    # Limpiar encabezados para evitar saltos de línea y espacios extra
    headers = [
        th.get_text(separator=" ", strip=True).replace('\n', ' ').replace('\r', '').strip()
        for th in table.find('thead').find_all('th')
    ]

    headers = [' '.join(h.split()) for h in headers]

    rows = []
    for tr in table.find('tbody').find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        rows.append(cells)

    df = pd.DataFrame(rows, columns=headers)
    df['SourceYear'] = year  
    return df

# Iterar sobre los años deseados
all_years = []
for year in range(2020, 2025):  # Cambia el rango si quieres más años
    print(f"Scraping {year}...")
    df_year = scrape_treasury_yield_curve(year)
    all_years.append(df_year)
    sleep(1)  # Evita sobrecargar el servidor

# Concatenar todo en un solo DataFrame
df_all = pd.concat(all_years, ignore_index=True)

# Limpiar columnas numéricas (solo columnas tipo object y que no sean 'SourceYear' ni 'Date')
df_all['Date'] = pd.to_datetime(df_all['Date'], errors='coerce')
for col in df_all.columns:
    if col not in ['Date', 'SourceYear']:
        if pd.api.types.is_object_dtype(df_all[col]):
            df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

df_all_pr = pd.concat([df_all.iloc[:, [0]], df_all.iloc[:, 12:]], axis=1)

# Guardar como CSV
# df_all_pr.to_csv('yield_curve_history.csv', index=False)

#df_all_pr

# Mostrar el DataFrame final
#display(df_all_pr)


Scraping 2020...
Scraping 2021...
Scraping 2022...
Scraping 2023...
Scraping 2024...


,Date,1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,SourceYear
0,2020-01-02,1.53,NaN,1.55,1.54,NaN,1.57,1.56,1.58,1.59,1.67,1.79,1.88,2.19,2.33,2020
1,2020-01-03,1.52,NaN,1.55,1.52,NaN,1.55,1.55,1.53,1.54,1.59,1.71,1.80,2.11,2.26,2020
2,2020-01-06,1.54,NaN,1.54,1.56,NaN,1.56,1.54,1.54,1.56,1.61,1.72,1.81,2.13,2.28,2020
3,2020-01-07,1.52,NaN,1.53,1.54,NaN,1.56,1.53,1.54,1.55,1.62,1.74,1.83,2.16,2.31,2020
4,2020-01-08,1.50,NaN,1.53,1.54,NaN,1.56,1.55,1.58,1.61,1.67,1.78,1.87,2.21,2.35,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,2024-12-24,4.44,NaN,4.44,4.40,4.38,4.30,4.24,4.29,4.36,4.43,4.52,4.59,4.84,4.76,2024
1247,2024-12-26,4.45,NaN,4.45,4.35,4.37,4.31,4.23,4.30,4.35,4.42,4.49,4.58,4.83,4.76,2024
1248,2024-12-27,4.44,NaN,4.43,4.31,4.35,4.29,4.20,4.31,4.36,4.45,4.53,4.62,4.89,4.82,2024
1249,2024-12-30,4.43,NaN,4.42,4.37,4.33,4.25,4.17,4.24,4.29,4.37,4.46,4.55,4.84,4.77,2024


In [8]:
df_all_pr.to_excel('datos_aexcel.xlsx')